In [1]:
import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
from random import randrange
# from torchsummary import summary
import os
import datetime

import sys
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
epochs = 100
batch_size = 64
width_fig = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
trainset = torch.load('../data/dataset_10steps_4ch.pt')

In [4]:
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

In [5]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Input size: [batch, 3, 100, 100]
        # Output size: [batch, 3, 100, 100]
        self.encoder = nn.Sequential(
            nn.Conv2d(4, 16, 3, stride=1, padding=1),
            # nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.Conv2d(16, 16, 5, stride=1, padding=1),
            # nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.Conv2d(16, 16, 7, stride=1, padding=1),
            # nn.BatchNorm2d(32),
            nn.LeakyReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, 16, 7, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(16, 16, 5, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(16, 3, 3, stride=1, padding=1),
        )

    def forward(self, x):
        # print(x.size())
        wall = x[:, -1]
        # print(wall.size())
        x = self.encoder(x)
        x = self.decoder(x)
        # print(x.size())
        x = torch.cat((x, wall.reshape(len(x), 1, width_fig, width_fig)), 1)
        # print(x.size())
        return x

In [6]:
# model = Autoencoder()
# model(torch.rand((64,4,100,100)))

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(4, 6, 9, stride=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 8, 7, stride=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout()
        self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        # print(x.size())
        x = self.pool(F.relu(self.conv1(x)))
        # print(x.size())
        x = self.pool(F.relu(self.conv2(x)))
        # print(x.size())
        x = torch.flatten(x, start_dim=1) # flatten all dimensions except batch
        # print(x.size())
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        # print(x.size())
        x = F.sigmoid(self.fc2(x))
        # print(x.size())
        return x

In [8]:
class StepAE(nn.Module):
    def __init__(self):
        """Variational Auto-Encoder Class"""
        super(StepAE, self).__init__()
        
        self.enc = Autoencoder()

    def step(self, x): # For actual run after training
        x = self.enc(x)
        return x

    def forward(self, x): # For training
        # Encode x to z
        recon1 = self.enc(x)
        recon2 = self.enc(recon1)
        recon3 = self.enc(recon2)
        recon4 = self.enc(recon3)
        # recon5 = self.enc(recon4)
        
        return recon1, recon2, recon3, recon4

# Training

In [9]:
disc_history_real = []
disc_history_fake = []
rme_history = []

In [10]:
# https://uos-deep-learning.tistory.com/16
def calc_gradient_penalty(netD, real_data, generated_data):
    # GP strength
    LAMBDA = 5

    b_size = real_data.size()[0]

    # Calculate interpolation
    alpha = torch.rand(b_size, 1, 1, 1)
    alpha = alpha.expand_as(real_data)
    alpha = alpha.cuda()

    interpolated = alpha * real_data.data + (1 - alpha) * generated_data.data
    interpolated = torch.autograd.Variable(interpolated, requires_grad=True)
    interpolated = interpolated.cuda()

    # Calculate probability of interpolated examples
    prob_interpolated = netD(interpolated)

    # Calculate gradients of probabilities with respect to examples
    gradients = torch.autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                           grad_outputs=torch.ones(prob_interpolated.size()).cuda(),
                           create_graph=True, retain_graph=True)[0]

    # Gradients have shape (batch_size, num_channels, img_width, img_height),
    # so flatten to easily take norm per example in batch
    gradients = gradients.view(b_size, -1)

    # Derivatives of the gradient close to 0 can cause problems because of
    # the square root, so manually calculate norm and add epsilon
    gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

    # Return gradient penalty
    return LAMBDA * ((gradients_norm - 1) ** 2).mean()

In [11]:
def train(model, discnet, train_loader, epochnow):
    model.train()
    discnet.train()
    avg_loss = 0
    for step, (x0, _, x1, _, x2, _, x3, _, x4, _) in enumerate(train_loader):
        # noisy_x = x + train_noise_level * torch.randn(*x.shape)
        # noisy_x = np.clip(noisy_x, 0., 1.)
        del _

        x0 = x0.permute(0, 3, 1, 2)
        x1 = x1.permute(0, 3, 1, 2)
        x2 = x2.permute(0, 3, 1, 2)
        x3 = x3.permute(0, 3, 1, 2)
        x4 = x4.permute(0, 3, 1, 2)

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z))
        # model.eval()
        discnet.train()
        discnet.zero_grad()
        optimizerD.zero_grad()
        r1, r2, r3, r4 = model(x0.cuda())
        
        # All real
        label_size = x1.size(0)
        real_label = 1
        label = torch.full((label_size,), real_label, dtype=torch.float, device=device)       
        for idx, x_ in enumerate([x1, x2, x3, x4]):
            output = discnet(x_.cuda()).view(-1)
            if idx == 0:
                errD_real = criterionD(output, label)
            else:
                errD_real += criterionD(output, label)
        errD_real.backward()
        out_real = output.detach().cpu().numpy()

        # All fake
        fake_label = 0
        label.fill_(fake_label)
        for idx, r_ in enumerate([r1, r2, r3, r4]):
            output = discnet(r_.detach().contiguous()).view(-1)
            if idx == 0:
                errD_fake = criterionD(output, label)
            else:
                errD_fake += criterionD(output, label)
        errD_fake.backward()

        grad_penalty = calc_gradient_penalty(netD, x1.cuda(), r1)
        grad_penalty += calc_gradient_penalty(netD, x2.cuda(), r2)
        grad_penalty += calc_gradient_penalty(netD, x3.cuda(), r3)
        grad_penalty += calc_gradient_penalty(netD, x4.cuda(), r4)
        grad_penalty = 1*grad_penalty
        grad_penalty.backward()

        optimizerD.step()
        del r1, r2, r3, r4

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        # model.train()
        discnet.eval()
        optimizer.zero_grad()
        model.zero_grad()

        r1, r2, r3, r4 = model(x0.cuda())
        label.fill_(real_label)
        for idx, r_ in enumerate([r1, r2, r3, r4]):
            output = discnet(r_.detach().contiguous()).view(-1)
            if idx == 0:
                errG = (idx+1)*criterionD(output, label)
            else:
                errG += (idx)*criterionD(output, label)

        loss1 = criterion(r1, x1.cuda())
        loss2 = criterion(r2, x2.cuda())
        loss3 = criterion(r3, x3.cuda())
        loss4 = criterion(r4, x4.cuda())
        # loss5 = criterion(r5, x5.cuda())
        loss = 10*(loss1 + loss2 + loss3 + loss4) + errG
        
        del x0, x1, x2, x3, x4, r1, r2, r3, r4

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_loss += loss.item()
        loss_history.append(loss.item())

        if step%10 == 0:
            print("step{}, loss = {}, real = {}, fake = {}".format(step, loss, np.mean(out_real), (output.mean().item())), errG.mean().item())
            disc_history_real.append([out_real])
            disc_history_fake.append([output.detach().cpu().numpy()])

        if step%100 == 0:
            # sys.stdout = open(os.devnull, 'w')
            figplot(epochnow, step)
            torch.save(model_origin.state_dict(), "../data/model/" + timestampStr + "/wider_stepwise_ep{}_{}.pt".format(epochnow, step))
            torch.save(netD.state_dict(), "../data/model/" + timestampStr + "/wider_discnet_ep{}_{}.pt".format(epochnow, step))
            torch.save(model_origin.state_dict(), "latest_net_wdisc_4ch_wider.pt".format(epochnow, step))
            torch.save(netD.state_dict(), "latest_disc_wdisc_4ch_wider.pt".format(epochnow, step))
            np.save("../data/model/" + timestampStr +"/disc_history_fake", disc_history_fake)
            np.save("../data/model/" + timestampStr +"/disc_history_real", disc_history_real)
            np.save("../data/model/" + timestampStr +"/loss_history", loss_history)
            # sys.stdout = sys.__stdout__

    return avg_loss / len(train_loader)

In [12]:
netD = Discriminator().to(device)
optimizerD = optim.Adam(netD.parameters(), lr=0.0001)
criterionD = nn.BCEWithLogitsLoss()

In [13]:
model_origin = StepAE().to(device)
criterion = nn.MSELoss()
loss_history = []

In [14]:
oneloader = DataLoader(trainset, batch_size=1, shuffle=True)
torch.cuda.empty_cache()

def figplot(epochnow, step):
    model_origin.eval()
    dataiter = iter(oneloader)
    images, _, _, _, _, _, _, _, _, _ = dataiter.next()
    del _

    recon = model_origin.step(images.permute(0, 3, 1, 2).cuda())

    # del _
    torch.cuda.empty_cache()
    # get sample outputs

    steps = 100
    skip = int(steps/10)
    
    plt.figure(figsize=(6, 24))
    ax=plt.subplot(12,1,1)
    plt.imshow(images.permute(0,3,1,2).view(1,4,100,100).cpu()[0][:-1].permute(1,2,0))
    ax=plt.subplot(12,1,2)
    output = recon.detach().view(1, 4, 100, 100).cpu()[0][:-1].permute(1,2,0)
    plt.imshow(output)
    ii = 0
    for i in range(steps):
        recon = model_origin.step(recon)
        # del _
        
        if i%skip == 0:
            ii+=1
            ax = plt.subplot(12, 1, ii + 2)
            # print(i)
            # output is resized into a batch of iages
            # use detach when it's an output that requires_grad
            output = recon.detach().view(1, 4, 100, 100).cpu()[0][:-1].permute(1, 2, 0)
            # print(output.min(), output.max())
            plt.imshow(output)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            del output
            torch.cuda.empty_cache()
    plt.savefig("../data/model/"+timestampStr+"/fig_{}_{}.png".format(epochnow, step))
    torch.save(images, "../data/model/"+timestampStr+"/input_{}_{}.pt".format(epochnow,step))
    del images
    del recon
    del dataiter

In [15]:
optimizer = torch.optim.Adam(model_origin.parameters(), lr=0.001)

import datetime
dateTimeObj = datetime.datetime.now()
timestampStr = dateTimeObj.strftime("%d_%b_%Y_%H_%M")
if timestampStr not in os.listdir("../data/model/"):
    os.mkdir("../data/model/"+timestampStr)

for epoch in range(1, epochs+1):
    epoch_loss = train(model=model_origin, discnet=netD, train_loader=trainloader, epochnow=epoch)
    print("[Epoch {}] loss:{}".format(epoch, epoch_loss))
    torch.save(model_origin.state_dict(), "../data/model/" + timestampStr + "/new_stepwise_ep{}.pt".format(epoch))
    torch.save(netD.state_dict(), "../data/model/" + timestampStr + "/new_discnet_ep{}.pt".format(epoch))

c:\Users\sbin_LAB\.conda\envs\torch\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


step0, loss = 10.957413673400879, real = 0.5578157305717468, fake = 0.5553172826766968 3.1748390197753906


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


step10, loss = 7.770522117614746, real = 0.5589277744293213, fake = 0.5599721670150757 3.165239095687866
step20, loss = 8.947179794311523, real = 0.5555657744407654, fake = 0.5479027032852173 3.1935133934020996
step30, loss = 8.750725746154785, real = 0.5550981163978577, fake = 0.5440421104431152 3.202759265899658
step40, loss = 7.732100486755371, real = 0.5545464754104614, fake = 0.5494291186332703 3.189897298812866
step50, loss = 5.83929967880249, real = 0.5511781573295593, fake = 0.5466221570968628 3.197788715362549
step60, loss = 5.49272346496582, real = 0.5388669371604919, fake = 0.5342454314231873 3.2276177406311035
step70, loss = 5.490797996520996, real = 0.5366247892379761, fake = 0.5278217792510986 3.244633674621582
step80, loss = 5.3556623458862305, real = 0.5240291357040405, fake = 0.5176575183868408 3.2711684703826904
step90, loss = 5.338774681091309, real = 0.5184926390647888, fake = 0.5106218457221985 3.2897815704345703


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

step1310, loss = 4.590946674346924, real = 0.18257758021354675, fake = 0.16201183199882507 4.306717395782471
step1320, loss = 4.652841567993164, real = 0.19764336943626404, fake = 0.1612866371870041 4.309396266937256
step1330, loss = 4.687500953674316, real = 0.18221253156661987, fake = 0.1622997522354126 4.305849075317383
step1340, loss = 4.631260871887207, real = 0.18998309969902039, fake = 0.16112536191940308 4.310258388519287
step1350, loss = 4.5829644203186035, real = 0.1850011646747589, fake = 0.16380810737609863 4.301294326782227
step1360, loss = 4.794493198394775, real = 0.1954890787601471, fake = 0.16574576497077942 4.295608997344971
step1370, loss = 4.615091323852539, real = 0.19108612835407257, fake = 0.16107457876205444 4.310466766357422
step1380, loss = 4.635688304901123, real = 0.17677515745162964, fake = 0.16443192958831787 4.298809051513672
step1390, loss = 4.586197853088379, real = 0.1791064739227295, fake = 0.1584416925907135 4.31856632232666


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


step1400, loss = 5.131313323974609, real = 0.19919538497924805, fake = 0.1656324565410614 4.296235084533691


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


step1410, loss = 4.612605094909668, real = 0.16924631595611572, fake = 0.16648922860622406 4.293083667755127
step1420, loss = 4.6121368408203125, real = 0.1742691993713379, fake = 0.16260561347007751 4.305174350738525
step1430, loss = 4.643726348876953, real = 0.17680566012859344, fake = 0.16018345952033997 4.313112258911133


In [ ]:
plt.plot(loss_history)
plt.show()
torch.cuda.empty_cache()

In [ ]:
# torch.cuda.empty_cache()
# model_origin = StepAE().to(device)
# model_origin.load_state_dict(torch.load("cifar10.pt"))
model_origin.eval()

# Visualize

In [ ]:
model_origin = StepAE().to(device)
model_origin.load_state_dict(torch.load("../data/model/27_May_2022_18_52/new_stepwise_ep5.pt"))
model_origin.eval()